# LeNet-5

    '''
    参数设置参考Justin 598WI2022课件，非原lecun论文
    参数量相对较多
    '''

<img src="./lenet.png" alt="LeNet-5 Image" width="800">

In [1]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_cv


In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

from sgd_cv.model import LeNet5

## 测试模型

In [ ]:
# 测试模型结构
model = LeNet5(num_classes=10)
print(model)

# 随机生成一个批次的输入 (MNIST 图像大小: 1x28x28)
input_tensor = torch.randn(1, 1, 28, 28)
output = model(input_tensor)

print(f"输入张量大小: {input_tensor.shape}")
print(f"输出张量大小: {output.shape}")  # 应为 [1, 10]

## 训练模型

In [ ]:
# 数据集加载
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(
    root='./data/MNIST/', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data/MNIST/', train=False, transform=transform, download=True
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
# 训练
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# 测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")